In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline                                           
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV

# Load microbiome metadata, composition and SCFA data

In [2]:
# load meta data
df_meta = pd.read_csv('meta_data.csv', index_col=0)
df_meta = df_meta[df_meta.Diet=='Inulin']
df_meta = df_meta[df_meta.Day != 0] # remove day 0 samples

# load SCFA data
df_scfa = pd.read_csv('SCFA.csv', index_col=0)

# load species abundance
df_bac = pd.read_csv('quantitative_abundance_species.csv', index_col=0)

# find samples present in all these tables
shared_samples = list(set(df_meta.index).intersection(df_scfa.index).intersection(df_bac.index))
df_meta = df_meta.loc[shared_samples]
df_scfa = df_scfa.loc[shared_samples]
df_bac = df_bac.loc[shared_samples]

# remove species that are constant across all samples
df_bac = df_bac[list(df_bac.std()[df_bac.std()>0].index)]

# Train Random Forest model and predict

## intrapolation (using a subset of mice from all vendors as training and the other subset as test)

In [3]:
results = []

for k,group_to_exclude in enumerate(['A','B','C','D']):

    # split train/test data
    mice_to_keep = list(set(df_meta[df_meta.RandomizedGroup!=group_to_exclude].MiceID))
    samples_to_keep = list(set(df_meta[df_meta.MiceID.isin(mice_to_keep)].index))
    mice_to_exclude = list(set(df_meta[df_meta.RandomizedGroup==group_to_exclude].MiceID))
    samples_to_exclude = list(set(df_meta[df_meta.MiceID.isin(mice_to_exclude)].index))
    xdata_train = np.asarray(df_bac.loc[samples_to_keep].values)
    xdata_test = np.asarray(df_bac.loc[samples_to_exclude].values)
    
    # run random forest regression
    for scfa in ['Acetate','Propionate','Butyrate']:                
        ydata_train = np.asarray(df_scfa.loc[samples_to_keep, scfa])
        ydata_test = np.asarray(df_scfa.loc[samples_to_exclude, scfa])

        # make a pipeline using standardscaler for transformation, lasso for feature selection, random forest for prediction
        param_grid = {
            'selectfrommodel__estimator__alpha':[10**v for v in [-4,-3,-2,-1,0]], # too large alpha will produce a null model (all features are 0)
            'randomforestregressor__max_features':['auto','sqrt','log2',0.16,0.32,0.64],
            'randomforestregressor__max_depth':[2,4,8,16],
            'randomforestregressor__min_samples_split':[2,4,8,16],
            'randomforestregressor__min_samples_leaf':[1,2,4]
        }
        
        # train RF model
        clf1 = linear_model.Lasso(tol=1e-5,positive=True,random_state=0,max_iter=1000000)
        clf2 = RandomForestRegressor(n_estimators=2000,random_state=0,oob_score=True)
        pipe = make_pipeline(StandardScaler(), SelectFromModel(clf1, threshold=1e-5), clone(clf2))  
        CV = GridSearchCV(pipe, param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=2)
        CV.fit(xdata_train, ydata_train)

        print('Intrapolation, group %s, %s, best score and parameter combination = '%(group_to_exclude, scfa))
        print(CV.best_score_)    
        print(CV.best_params_)    
        print('\n')

        # predict train and test data
        ydata_train_predicted = CV.predict(xdata_train)
        ydata_test_predicted = CV.predict(xdata_test)
        for sample_, obs_, pred_ in zip(samples_to_keep, ydata_train, ydata_train_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['intrapolation', scfa, group_to_exclude, 'train', sample_, day_, obs_, pred_])
        for sample_, obs_, pred_ in zip(samples_to_exclude, ydata_test, ydata_test_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['intrapolation', scfa, group_to_exclude, 'test', sample_, day_, obs_, pred_])

## extrapolation (using all mice from a subset of vendors as training and the other subset as test)

In [4]:
for k,vendor_to_exclude in enumerate(['Beijing','Guangdong','Hunan','Shanghai']):
        
    # split train/test data
    mice_to_keep = list(set(df_meta[df_meta.Vendor!=vendor_to_exclude].MiceID))
    samples_to_keep = list(set(df_meta[df_meta.MiceID.isin(mice_to_keep)].index))
    mice_to_exclude = list(set(df_meta[df_meta.Vendor==vendor_to_exclude].MiceID))
    samples_to_exclude = list(set(df_meta[df_meta.MiceID.isin(mice_to_exclude)].index))
    xdata_train = np.asarray(df_bac.loc[samples_to_keep].values)
    xdata_test = np.asarray(df_bac.loc[samples_to_exclude].values)
    
    # run random forest regression
    for scfa in ['Acetate','Propionate','Butyrate']:                
        ydata_train = np.asarray(df_scfa.loc[samples_to_keep, scfa])
        ydata_test = np.asarray(df_scfa.loc[samples_to_exclude, scfa])

        # make a pipeline using standardscaler for transformation, lasso for feature selection, random forest for prediction
        param_grid = {
            'selectfrommodel__estimator__alpha':[10**v for v in [-4,-3,-2,-1,0]], # too large alpha will produce a null model (all features are 0)
            'randomforestregressor__max_features':['auto','sqrt','log2',0.16,0.32,0.64],
            'randomforestregressor__max_depth':[2,4,8,16],
            'randomforestregressor__min_samples_split':[2,4,8,16],
            'randomforestregressor__min_samples_leaf':[1,2,4]
        }
        
        clf1 = linear_model.Lasso(tol=1e-5,positive=True,random_state=0,max_iter=1000000)
        clf2 = RandomForestRegressor(n_estimators=2000,random_state=0,oob_score=True)
        pipe = make_pipeline(StandardScaler(), SelectFromModel(clf1, threshold=1e-5), clone(clf2))  
        CV = GridSearchCV(pipe, param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=2)
        CV.fit(xdata_train, ydata_train)

        print('Extrapolation, vendor %s, %s, best score and parameter combination = '%(vendor_to_exclude, scfa))
        print(CV.best_score_)    
        print(CV.best_params_)    
        print('\n')   

        # predict train and test data
        ydata_train_predicted = CV.predict(xdata_train)
        ydata_test_predicted = CV.predict(xdata_test)

        for sample_, obs_, pred_ in zip(samples_to_keep, ydata_train, ydata_train_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['extrapolation', scfa, vendor_to_exclude, 'train', sample_, day_, obs_, pred_])
        for sample_, obs_, pred_ in zip(samples_to_exclude, ydata_test, ydata_test_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['extrapolation', scfa, vendor_to_exclude, 'test', sample_, day_, obs_, pred_])

## save interpolation and extrapolation results to file

In [5]:
df_prediction = pd.DataFrame(results, columns=['PerturbationType','SCFA','Permutation','PredictionType','SampleID','Day','ObservedValue','PredictedValue'])
df_prediction.to_csv('rf_prediction.csv')